In [1]:
from dhlab.module_update import update, css
import dhlab.nbtext as nb
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import dhlab.token_map as tm

%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


In [37]:
from typing import Iterable
import pandas as pd 
import dhlab as dh
from dhlab import Corpus, NER, Models
from dhlab.text.corpus import urnlist
from datetime import datetime
from importlib.metadata import version

version("dhlab")

'2.7.1.dev2+ge1d4667'

# Persongalleri i forfatterskap

## 1. Bygg korpus
Først bygger vi et korpus over forfatterskapet til f.eks. Sigrid Undset

For å få et jevnt utvalg fra tidsperioden forfatteren har utgitt bøker i, henter vi et mindre antall fra hvert tiår og slår sammen korporaene til slutt. 

In [42]:
author = "Sigrid Undset"                                                    # Velg f.eks. forfatter
timestep = 10                                                               # intervall mellom perdiodene vi søker i 
first_publication = 1900                                                    # forsøksvis anslag av første tiår
last_publication = 1990
partial_limit = 10                                                          # Begrenset antall verk per tiår

corpus = dh.EmptyCorpus()                                                   # Start med tomt korpus som vi kan bygge opp


print("Antall publikasjoner i søksresultatet: ")

for decade in range(first_publication, last_publication, timestep):
    partial_corpus = dh.Corpus(
        author=author, 
        from_year=decade, 
        to_year=decade + timestep,
        limit=partial_limit
    )
    print(f"{decade}-{decade+timestep}: {partial_corpus.size}")
    if partial_corpus.size > 0:
        corpus.add(partial_corpus)                                      # Legg tiårets publikasjoner til korpuset bare dersom det er noen

print(f"Totalt antall unike publikasjoner i korpuset: {corpus.size}")


Antall publikasjoner i søksresultatet: 
1900-1910: 3
1910-1920: 8
1920-1930: 10
1930-1940: 10
1940-1950: 10
1950-1960: 10
1960-1970: 10
1970-1980: 10
1980-1990: 10
Totalt antall unike publikasjoner i korpuset: 81


In [43]:
df = corpus.corpus
df.head()              # Kikk på de første linjene i korpuset

,dhlabid,urn,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype
0,100577723,URN:NBN:no-nb_digibok_2008040204017,Fru Marta Oulie,"Undset , Sigrid",oai:nb.bibsys.no:998901868204702202,f968e130774818bc763ff7298fb70cab,,Kristiania,19070101,1907,Aschehoug,nob,,,,Uklassifisert,digibok
1,100505641,URN:NBN:no-nb_digibok_2007050912002,Den lykkelige alder,"Undset , Sigrid",oai:nb.bibsys.no:999612430584702202,b1068517b9d46ebb5e09ae61e0aa64ef,,Kristiania,19080101,1908,Aschehoug,nob,,,short story,Skjønnlitteratur,digibok
2,100505639,URN:NBN:no-nb_digibok_2007050812003,Fortællingen om Viga-Ljot og Vigdis,"Undset , Sigrid",oai:nb.bibsys.no:999110884824702202,bc57ad39def59311af88a6bdae325f47,,Kristiania,19090101,1909,Aschehoug,nob,,,,Uklassifisert,digibok
3,100505645,URN:NBN:no-nb_digibok_2007051012004,Fattige skjæbner,"Undset , Sigrid",oai:nb.bibsys.no:998900878734702202,77f9ab062592f0a0ba2bdc8beb6d8874,,Kristiania,19120101,1912,Aschehoug,nob,,,,Uklassifisert,digibok
4,100004162,URN:NBN:no-nb_digibok_2007042412001,Splinten av troldspeilet,"Undset , Sigrid",oai:nb.bibsys.no:990101760124702202,43552b3dbb4813dd34b810bf5494c997,,Kristiania,19170101,1917,Aschehoug,nob,,,fiction,Skjønnlitteratur,digibok


## 2. Navnegjenkjenning

Vi kan bruke heuristikk og regler eller statistisk trente modeller for å hente ut egennavn fra tekstene. 
Først ser vi hvilket utvalg av modeller vi har, før vi kjører en analyse per publikasjon. 

In [46]:
# 2. Kjør Navnegjenkjenning (eng: Named Entity Recognition / NER) på korpuset

# Velg modell 
ner_models = Models()
print(ner_models.models)   # utdata:  ['nb_core_news_lg', 'da_core_news_lg', 'nb_core_news_sm', 'dhlab']
model="dhlab"

# Hent URN fra korpuset
document_urns= urnlist(corpus) 

navneliste = []
for urn in document_urns: 
    navneliste.append(NER(urn, model))         # Hent ut navn

navn = pd.concat(navneliste, ignore_index=True)


['nb_core_news_lg', 'da_core_news_lg', 'nb_core_news_sm', 'dhlab']


In [ ]:
navn